In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import f1_score



In [2]:
raw_data = pd.read_csv('../Data/santander-customer-transaction-prediction/train.csv')
raw_data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
raw_data.dtypes.value_counts()

float64    200
object       1
int64        1
Name: count, dtype: int64

In [4]:
raw_data['target'].dtype

dtype('int64')

In [5]:
raw_data.isnull().sum()

ID_code    0
target     0
var_0      0
var_1      0
var_2      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 202, dtype: int64

In [6]:
raw_data.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [8]:
raw_data.drop('ID_code', axis=1, inplace=True)

In [9]:
x = raw_data.drop(['target'], axis=1)
y = raw_data['target']

In [10]:
x_train, x_test , y_train, y_test = train_test_split(x, y, test_size=0.2)

In [11]:
x_train, y_train = SMOTE(random_state=42).fit_resample(x_train, y_train)


In [12]:
y_train.value_counts()

target
0    143895
1    143895
Name: count, dtype: int64

In [13]:
# sc = StandardScaler()
# x_train = sc.fit_transform(x_train)

In [14]:
# pca = PCA(n_components=0.9)
# X_train_pca = pca.fit_transform(x_train)

In [15]:
# X_train_pca.shape

In [32]:
xgb_model_withPCA = xgb.XGBClassifier(
    booster='gbtree',                   
    objective='binary:logistic',                              
    learning_rate=0.5,
    n_estimators=500,                   
    subsample=1,                        
    colsample_bytree=1.0,                                                               
    random_state=42,
    gamma = 0,
    scale_pos_weight=len(y_train) / (2 * np.bincount(y_train)[1])          
    )          


In [33]:
xgb_model_withoutPCA = xgb.XGBClassifier(
    booster='gbtree',                   
    objective='binary:logistic',                              
    learning_rate=0.5,
    n_estimators=500,                   
    subsample=1,                        
    colsample_bytree=1.0,                                                               
    random_state=42,
    gamma = 0,
    scale_pos_weight=len(y_train) / (2 * np.bincount(y_train)[1])          
    )   

In [34]:

def f1_class_1(y_true, y_pred):
    return f1_score(y_true, y_pred, pos_label=1)

In [35]:
pipeline_with_pca = ImbPipeline([    
    ('scaler', StandardScaler()),         
    ('pca', PCA(n_components=0.9)),      
    ('xgb', xgb_model_withPCA)
])

In [36]:
pipeline_without_pca = ImbPipeline([    
    ('scaler', StandardScaler()),         
    ('pca', PCA(n_components=0.9)),      
    ('xgb', xgb_model_withoutPCA)
])

In [20]:
param_grid = {
    # 'xgb__learning_rate': [0.1, 0.3, 0.5, 1.0]
    # 'xgb__n_estimators': [500,1000]
    # 'xgb__max_depth': [3, 5, 7],
    # 'xgb__subsample': [0.8, 1.0]
    # 'xgb__colsample_bytree': [0.8, 1.0],
    'xgb__gamma': [0, 1, 5]
    # 'xgb__min_child_weight': [1, 3, 5]
}

In [21]:
# grid_search = GridSearchCV(
#     pipeline,
#     param_grid,
#     cv=5,                      
#     scoring=f1_class_1_scorer,        
#     n_jobs=-1,                 
#     verbose=1                 
# )


In [22]:
# grid_search.fit(x_train, y_train)

In [23]:
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

NameError: name 'grid_search' is not defined

In [37]:
pipeline_with_pca.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=0.9)),
                ('xgb',
                 XGBClassifier(base_score=None, booster='gbtree',
                               callbacks=None, colsample_bylevel=None,
                               colsample_bynode=None, colsample_bytree=1.0,
                               device=None, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.5,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [38]:
pipeline_without_pca.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=0.9)),
                ('xgb',
                 XGBClassifier(base_score=None, booster='gbtree',
                               callbacks=None, colsample_bylevel=None,
                               colsample_bynode=None, colsample_bytree=1.0,
                               device=None, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.5,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [26]:
score_without_PCA = pipeline_without_pca.score(x_test, y_test)

In [30]:
score_without_PCA

0.877375

In [39]:
score = pipeline_with_pca.score(x_test, y_test)
score

0.877375

In [40]:
y_pred_1 = pipeline_without_pca.predict(x_test)


In [41]:
y_pred_2 = pipeline_with_pca.predict(x_test)


In [ ]:
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.92      0.94      0.93     35903
           1       0.37      0.31      0.34      4097

    accuracy                           0.88     40000
   macro avg       0.65      0.63      0.64     40000
weighted avg       0.87      0.88      0.87     40000



In [ ]:
scores = cross_val_score(pipeline, x_train, y_train , scoring=f1_class_1_scorer)

In [ ]:
scores

NameError: name 'scores' is not defined

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred) 
cm

array([[33725,  2178],
       [ 2816,  1281]])